## Farm Fresh Foods - Capstone Project

In [1]:
import numpy as np # handles data

import pandas as pd # library for data exploration / analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #handles JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 
import requests #handles requests
from pandas.io.json import json_normalize # tranforms JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium #renders mapping

import csv

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [3]:
!wget -q -O 'newyork_neighborhoods.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded.')

Data downloaded.


In [4]:
with open('newyork_neighborhoods.json') as json_data:
    newyork_neighborhoods = json.load(json_data)

In [5]:
neighborhoods_data = newyork_neighborhoods['features']

In [6]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [7]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

nyneighborhood = pd.DataFrame(columns=column_names)

In [8]:
nyneighborhood

,Borough,Neighborhood,Latitude,Longitude


In [9]:
for data in neighborhoods_data:
    borough = newyork_neighborhoods = data['properties']['borough'] 
    newyork_neighborhoods = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nyneighborhood = nyneighborhood.append({'Borough': borough,
                                          'Neighborhood': newyork_neighborhoods,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
nyneighborhood.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Let's begin to explore the number of boroughs and neighborhoods within this data set.

In [11]:
print('This dataframe contains {} boroughs and {} neighborhoods.'.format(
        len(nyneighborhood['Borough'].unique()),
        nyneighborhood.shape[0]
    )
)

This dataframe contains 5 boroughs and 306 neighborhoods.


#### Now, we will use the geopy library to locate the lat and long of New York City, which will then be used to plot our dataset coordinates on a map.

In [12]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="Jupyter")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpic coordinates of New York City are {}, {}.'.format(latitude, longitude))

The geograpic coordinates of New York City are 40.7127281, -74.0060152.


In [13]:
map_NewYork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nyneighborhood['Latitude'], nyneighborhood['Longitude'], nyneighborhood['Borough'], nyneighborhood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NewYork)
map_NewYork

#### Now, we will proceed to scrape population and demographics data from the Wikipedia New York page